<a href="https://colab.research.google.com/github/KUrushi/toy_code/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
! pip install ml_collections

     |████████████████████████████████| 88 kB 6.2 MB/s 


In [1]:
import ml_collections
cfg = ml_collections.ConfigDict()
cfg.float_field = 12.6
cfg.integer_field = 123
cfg.anthoer_integer_field = 234
cfg.nested = ml_collections.ConfigDict()
cfg.nested.string_field = 'tom'

In [2]:
print(cfg.integer_field)
print(cfg['integer_field'])

try:
  cfg.integer_field = 'mo'
except TypeError as e:
  print(e)

cfg.float_field = 12
cfg.nested.string_field = u'bob'
print(cfg)

123
123
Could not override field 'integer_field' (reference). mo is of type <class 'str'> but should be of type <class 'int'>
anthoer_integer_field: 234
float_field: 12.0
integer_field: 123
nested: {string_field: bob}



# FrozenConfigDict
immutable, hashable type of configdict


In [3]:
import ml_collections

initial_dictionary = {
    'int': 1,
    'list': [1, 2],
    'tuple': (1, 2, 3),
    'set': {1, 2, 3, 4},
    'dict_tuple_list': {'tuple_list': ([1, 2], 3)}
}

cfg = ml_collections.ConfigDict(initial_dictionary)
frozen_dict = ml_collections.FrozenConfigDict(initial_dictionary)

print(frozen_dict.tuple)
print(frozen_dict.list)
print(frozen_dict.set)
print(frozen_dict.dict_tuple_list.tuple_list[0])

(1, 2, 3)
(1, 2)
frozenset({1, 2, 3, 4})
(1, 2)


In [5]:
frozen_cfg = ml_collections.FrozenConfigDict(cfg)
print(frozen_cfg == frozen_dict)
print(hash(frozen_cfg) == hash(frozen_dict))

True
True


In [6]:
try:
  frozen_dict.int = 2
except AttributeError as e:
  print(e)

FrozenConfigDict is immutable. Cannot call __setattr__().


In [7]:
thawed_frozen_cfg = ml_collections.ConfigDict(frozen_dict)
print(thawed_frozen_cfg == cfg)  # True
frozen_cfg_to_cfg = frozen_dict.as_configdict()
print(frozen_cfg_to_cfg == cfg)  # True

True
True


# FieldReferences and placeholders

FieldReferenceは、複数のフィールドで同じ値を使用する場合に便利です。また、遅延計算にも使用できます。

`placeholder()` をショートカットとして使用すると、デフォルト値が None の FieldReference (フィールド) を作成できます。これは、プログラムでオプションの構成フィールドを使用する場合に便利です。

In [8]:
import ml_collections
from ml_collections.config_dict import config_dict

placeholder = ml_collections.FieldReference(0)
cfg = ml_collections.ConfigDict()
cfg.placeholder = placeholder
cfg.optional = config_dict.placeholder(int)
cfg.nested = ml_collections.ConfigDict()
cfg.nested.placeholder = placeholder


In [9]:
try:
  cfg.optional = 'tom'
except TypeError as e:
  print(e)

Could not override field 'optional' (reference). tom is of type <class 'str'> but should be of type <class 'int'>


In [11]:
print(cfg)

nested: {placeholder: 0}
optional: null
placeholder: 0



In [12]:
cfg.optional = 12345
cfg.placeholder = 1

In [13]:
print(cfg)

nested: {placeholder: 1}
optional: 12345
placeholder: 1



In [15]:
import ml_collections
cfg = ml_collections.ConfigDict()
placeholder = ml_collections.FieldReference(0)
cfg.field1 = placeholder
print(cfg)
cfg.field2 = placeholder  # This field will be tied to cfg.field1.
print(cfg)
cfg.field3 = cfg.field1  
print(cfg)


{field1: 0}

{field1: 0, field2: 0}

{field1: 0, field2: 0, field3: 0}




FieldReferencesとプレースホルダー

FieldReferenceは、複数のフィールドで同じ値を使用する場合に便利です。また、遅延計算にも使用できます。

placeholder() をショートカットとして使用すると、デフォルト値が None の FieldReference (フィールド) を作成できます。これは、プログラムでオプションの構成フィールドを使用する場合に便利です。


In [16]:
import ml_collections
ref = ml_collections.FieldReference(1)
print(ref.get())

add_ten  = ref.get() + 10
add_ten_lazy = ref + 10

print(add_ten)
print(add_ten_lazy.get())

ref.set(5)
print(add_ten)
print(add_ten_lazy.get())

1
11
11
11
15


FieldReferenceがその元の値としてNoneを持っている場合、または何らかの操作がNoneの引数を持っている場合、遅延計算はNoneと評価されます。

また、ConfigDictのフィールドを遅延計算に使用することもできます。この場合、ConfigDict.get_ref()を使用してフィールドを取得した場合のみ、遅延計算が行われます。

In [17]:
import ml_collections

config = ml_collections.ConfigDict()
config.reference_field = ml_collections.FieldReference(1)
config.integer_field = 2
config.float_field = 2.5

# No lazy evaluatuations because we didn't use get_ref()
config.no_lazy = config.integer_field * config.float_field

# This will lazily evaluate ONLY config.integer_field
config.lazy_integer = config.get_ref('integer_field') * config.float_field

# This will lazily evaluate ONLY config.float_field
config.lazy_float = config.integer_field * config.get_ref('float_field')

# This will lazily evaluate BOTH config.integer_field and config.float_Field
config.lazy_both = (config.get_ref('integer_field') *
                    config.get_ref('float_field'))

config.integer_field = 3
print(config.no_lazy)  # Prints 5.0 - It uses integer_field's original value

print(config.lazy_integer)  # Prints 7.5

config.float_field = 3.5
print(config.lazy_float)  # Prints 7.0
print(config.lazy_both)  # Prints 10.5

5.0
7.5
7.0
10.5



遅延計算された値の変更

ConfigDict内の遅延計算された値は、通常の値と同じ方法で上書きすることができます。遅延計算に使われたFieldReferenceへの参照は失われ、参照グラフの下流のすべての計算は新しい値を使用します。


In [18]:
import ml_collections
config = ml_collections.ConfigDict()
config.reference = 1
config.reference_0 = config.get_ref('reference') + 10
config.reference_1 = config.get_ref('reference') + 20
config.reference_1_0 = config.get_ref('reference_1') + 100

print(config.reference)  # Prints 1.
print(config.reference_0)  # Prints 11.
print(config.reference_1)  # Prints 21.
print(config.reference_1_0)  # Prints 121.

config.reference_1 = 30

print(config.reference)  # Prints 1 (unchanged).
print(config.reference_0)  # Prints 11 (unchanged).
print(config.reference_1)  # Prints 30.
print(config.reference_1_0)  # Prints 130.

1
11
21
121
1
11
30
130


In [19]:
import ml_collections
from ml_collections.config_dict import config_dict

config = ml_collections.ConfigDict()
config.integer_field = 1
config.bigger_integer_field = config.get_ref('integer_field') + 10

try:
  # Raises a MutabilityError because setting config.integer_field would
  # cause a cycle.
  config.integer_field = config.get_ref('bigger_integer_field') + 2
except config_dict.MutabilityError as e:
  print(e)

Found cycle in reference graph.


# Advanced Usage

In [21]:
import ml_collections
config = ml_collections.ConfigDict()

config.float_field = 12.6
config.integer_field = 123
config.list_field = [0, 1, 2]

config.float_multiply_field = config.get_ref('float_field') * 3

print(config.float_multiply_field)

config.float_field = 10.0
print(config.float_multiply_field)


config.longer_list_field = config.get_ref('list_field') +  [3, 4, 5]
print(config.longer_list_field)


config.list_field = [-1]
print(config.longer_list_field)  # Prints [-1, 3, 4, 5]

# Both operands can be references
config.ref_subtraction = (
    config.get_ref('float_field') - config.get_ref('integer_field'))
print(config.ref_subtraction)  # Prints -113.0

config.integer_field = 10
print(config.ref_subtraction) 

37.8
30.0
[0, 1, 2, 3, 4, 5]
[-1, 3, 4, 5]
-113.0
0.0


In [22]:
import ml_collections

dict_1 = {'list': [1, 2]}
dict_2 = {'list': (1, 2)}
cfg_1 = ml_collections.ConfigDict(dict_1)
frozen_cfg_1 = ml_collections.FrozenConfigDict(dict_1)
frozen_cfg_2 = ml_collections.FrozenConfigDict(dict_2)

# True because FrozenConfigDict converts lists to tuples
print(frozen_cfg_1.items() == frozen_cfg_2.items())
# False because == distinguishes the underlying difference
print(frozen_cfg_1 == frozen_cfg_2)

# False because == distinguishes these types
print(frozen_cfg_1 == cfg_1)
# But eq_as_configdict() treats both as ConfigDict, so these are True:
print(frozen_cfg_1.eq_as_configdict(cfg_1))
print(cfg_1.eq_as_configdict(frozen_cfg_1))

True
False
False
True
True


In [23]:
import copy
import ml_collections

cfg = ml_collections.ConfigDict()
cfg.integer_field = 123

# Locking prohibits the addition and deletion of new fields but allows
# modification of existing values.
cfg.lock()
try:
  cfg.integer_field = 124  # Raises AttributeError and suggests valid field.
except AttributeError as e:
  print(e)
with cfg.unlocked():
  cfg.integer_field = 1555  # Works fine too.

# Get a copy of the config dict.
new_cfg = copy.deepcopy(cfg)
new_cfg.integer_field = -123  # Works fine.

print(cfg)

{integer_field: 1555}



コンフィグフラグ

このライブラリは、設定ファイルを扱うために absl.flags にフラグ定義を追加します。このライブラリは absl.flags をラップしていないので、標準的なフラグ定義を設定ファイルのフラグと一緒に使う場合は、ユーザは absl.flags もインポートしなければなりません。

現在、このモジュールは2つの新しいフラグタイプ、すなわち、設定を生成するPythonファイルへのパスを受け入れるDEFINE_config_fileと、設定を直接受け入れるDEFINE_config_dictを追加しています。DEFINE_config_dict は設定を直接受け取ることができます。設定は辞書のような構造体(ConfigDict を参照)で、そのネストされた要素は特別なコマンドラインフラグを使ってオーバーライドすることができます。詳細については，以下の例を参照してください．

www.DeepL.com/Translator（無料版）で翻訳しました。

In [24]:
%%writefile script.py
from absl import app
from absl import flags

from ml_collections.config_flags import config_flags

FLAGS = flags.FLAGS
config_flags.DEFINE_config_file('my_config')

def main(_):
  print(FLAGS.my_config)

if __name__ == "__main__":
  app.run(main)

Writing script.py


In [26]:
%%writefile config.py

import ml_collections
def get_config():
  config = ml_collections.ConfigDict()
  config.field1 = 1
  config.field2 = 'tom'
  config.nested = ml_collections.ConfigDict()
  config.nested.field = 2.23
  config.tuple = (1, 2, 3)
  return config

Writing config.py


In [27]:
%%bash
python script.py --my_config=config.py \
                 --my_config.field1=8 \
                 --my_config.nested.field=2.1 \
                 --my_config.tuple='(1, 2, (1, 2))'

field1: 8
field2: tom
nested: {field: 2.1}
tuple: !!python/tuple
- 1
- 2
- !!python/tuple [1, 2]



In [28]:
%%writefile script.py
from absl import app
from absl import flags

import ml_collections
from ml_collections.config_flags import config_flags

config = ml_collections.ConfigDict()
config.field = 1
config.field2 = 'tom'
config.nested = ml_collections.ConfigDict()
config.nested.field = 2.23
config.tuple = (1, 2, 3)


FLAGS = flags.FLAGS
config_flags.DEFINE_config_dict('my_config', config)

def main(_):
  print(FLAGS.my_config)


if __name__ == "__main__":
  app.run()

Overwriting script.py


In [30]:
from absl import logging

logging.info("Interesting Stuff")
logging.info(f'Interesting Stuff with Arguments: {42}')

logging.set_verbosity(logging.INFO)
logging.log(logging.DEBUG, "This wil *not* be printed")
logging.set_verbosity(logging.DEBUG)
logging.debug("This will be printed")

logging.warning("Worrying Stuff")
logging.error("Alarming Stuff")
logging.fatal("AAAAHHHHH!!!!")

INFO:absl:Interesting Stuff
INFO:absl:Interesting Stuff with Arguments: 42
DEBUG:absl:This will be printed
ERROR:absl:Alarming Stuff
CRITICAL:absl:AAAAHHHHH!!!!


In [33]:
try:
  1 / 0
except ZeroDivisionError as e:
  logging.exception(
      "わからん 😴"
  )

ERROR:absl:わからん 😴
Traceback (most recent call last):
  File "<ipython-input-33-86d0955c8379>", line 2, in <module>
    1 / 0
ZeroDivisionError: division by zero


In [34]:
import codecs
import contextlib
import copy
import os
import re
import subprocess
import sys
import tempfile
import unittest

from absl import app
from absl import flags
from absl._enum_module import enum
from absl.testing import _bazelize_command
from absl.testing import absltest
from absl.testing import flagsaver
from absl.tests import app_test_helper
import mock
import six

ModuleNotFoundError: ignored